# 2.1 Individual Model 1-wenkanw

In [1]:
import numpy as np
import pandas as pd
import torch
from torch import optim, nn
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os
from dataset import create_train_test_file_list, Person_MealsDataset, balance_data_indices


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Device: ",device,"Device Count: ", torch.cuda.device_count(), "Device Name: ",torch.cuda.get_device_name()  )
    print("Torch version:", torch.__version__)
device

Device:  cuda Device Count:  2 Device Name:  Tesla V100-PCIE-16GB
Torch version: 1.7.1


device(type='cuda')

In [2]:
import sys

### imports
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # Turn off TensorFlow logging
import tensorflow.compat.v1 as tf # maintain compatibility with TensorFlow 2.2.0

import keras
# from tensorflow.compat.v1.keras import backend as K # changed for compatibility with TensorFlow 2.2.0
import numpy as np
import pandas as pd
import random
#from sklearn.metrics import classification_report, confusion_matrix
from datetime import datetime
import loadfile
import addons

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape, Activation
from keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D
from keras.utils import np_utils
from keras.regularizers import l1
from keras.models import load_model, save_model

from tensorflow.compat.v1.keras import backend as K
import numpy as np

shimmer_global_mean = [-0.012359981,-0.0051663737,0.011612018,
                        0.05796114,0.1477952,-0.034395125 ]

shimmer_global_stddev = [0.05756385,0.040893298,0.043825723,
                        17.199743,15.311142,21.229317 ]

shimmer_trended_mean = [-0.000002,-0.000002,-0.000000,
                0.058144,0.147621,-0.033260 ]

shimmer_trended_stddev = [0.037592,0.034135,0.032263,
                17.209038,15.321441,21.242532 ]

all_zero_means = [0,0,0,0,0,0]

meanvals = all_zero_means
stdvals = shimmer_trended_stddev


random_seed  = 1000
batch_size = 128

In [3]:
# Load my models and functions
from dataset import create_train_test_file_list, Person_MealsDataset, balance_data_indices
from utils import *
from model import *


In [4]:
EPOCHS = 30
winmin = 3 
stridesec = 15

print_settings(winmin,stridesec, EPOCHS)

*****************************************************************

Execution Started at 12/20/2020, 21:09:48
WindowLength: 3.00 min (2700 datum)	Slide: 15 (225 datum)	Epochs:30



In [5]:
person = "wenkanw"
meal_data = Person_MealsDataset(person_name= person, file_name = "all_files_list", winmin = winmin,stridesec = stridesec)
samples,labels =  meal_data.data_indices, meal_data.labels
# split train set and test set
train_indices, test_indices = split_train_test_indices(X= [i for i in range(len(labels))],
                                                        y = labels, test_size = 0.2,
                                                       random_seed = random_seed)
# balance train set
trainset_labels = labels[train_indices]
train_indices_balanced = balance_data_indices(trainset_labels,data_indices= train_indices,mode="under", shuffle=True,random_state = random_seed,replace= False)


Loading Dataset ...
Loading File:  ../data/IndividualData/wenkanw-data/10-10-20/10-10-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/10-12-20/10-12-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/10-13-20/10-13-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/10-14-20/10-14-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/10-3-20/10-3-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/10-9-20/10-10-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/10-9-20/10-9-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/11-14-20/11-14-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/11-16-20/11-16-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/11-18-20/11-18-20.shm
Loading File:  ../data/IndividualData/wenkanw-data/11-21-20/Dinner/Dinner.shm
Loading File:  ../data/IndividualData/wenkanw-data/11-21-20/lunch/lunch.shm
Loading File:  ../data/IndividualData/wenkanw-data/11-22-20/Dinner/Dinner.shm
Loading File:  ../data/

In [6]:
train_set_balanced = torch.utils.data.Subset(meal_data, train_indices_balanced)
test_set = torch.utils.data.Subset(meal_data, test_indices)

train_loader = torch.utils.data.DataLoader(train_set_balanced,batch_size=batch_size, shuffle=True,num_workers=2)
test_loader = torch.utils.data.DataLoader(test_set ,batch_size=batch_size, shuffle=True,num_workers=2)

print("Data Loader Created")

Data Loader Created


In [7]:
torch.manual_seed(random_seed)

# split validation set
balanced_trainset_labels = labels[train_indices_balanced]
train_indices, valid_indices = split_train_test_indices(X= train_indices_balanced,
                                                        y = balanced_trainset_labels, test_size = 0.2,
                                                       random_seed = random_seed)

valid_set_balanced = torch.utils.data.Subset(meal_data, valid_indices)
valid_loader = torch.utils.data.DataLoader(valid_set_balanced,batch_size=batch_size, shuffle=True,num_workers=2)

# train model
input_shape = (train_set_balanced[0][0].shape[0], train_set_balanced[0][0].shape[1])
model_1 = Discriminator_ResNet( ngpu=1, input_shape =input_shape , out_fea = 1)
_ = model_1(torch.rand((1, input_shape[0],input_shape[1])))
model_1.to(device)
criterion_1 = nn.BCEWithLogitsLoss()
optimizer_1 = optim.Adam(model_1.parameters(),lr=0.01,  weight_decay=0.0)
lrscheduler_1 = optim.lr_scheduler.ReduceLROnPlateau(optimizer_1, mode='min',patience= 2, factor = 0.1,threshold=0.01)
dataloader = (train_loader, valid_loader)
model_1, best_model_1,val_score_1,loss_ls_1, train_acc_ls_1, valid_acc_ls_1 = train_model(model_1,dataloader, optimizer_1, 
                                                                    criterion_1, lrscheduler_1, device= device,
                                                                    n_epochs=30, patience = 5, l1_enabled=False,
                                                                    checkpoint_name ="../models/wenkanw_models/checkpoint_model_resnet.pt" )

Train set size: 2358, with 1179 positive samples and 1179 negative samples
Test set size: 590, with 295 positive samples and 295 negative samples
Training set batch amounts: 24
Test set : 5
Start Training..


KeyboardInterrupt: 

In [13]:
best_model_1.eval()
acc, recall = eval_model(best_model_1, test_loader,device)
print("Test Accuracy:", acc)

Test Accuracy: 88.41097224476546


In [8]:
# Test to check if random generated indices can be reproducted
# ls = balance_data_indices(traininglabels,mode="under", shuffle=True,random_state = random_seed)
# ls ==train_shuffledUnderSampledBalancedIndices

In [7]:
torch.manual_seed(random_seed)

# split validation set
balanced_trainset_labels = labels[train_indices_balanced]
train_indices, valid_indices = split_train_test_indices(X= train_indices_balanced,
                                                        y = balanced_trainset_labels, test_size = 0.2,
                                                       random_seed = random_seed)

valid_set_balanced = torch.utils.data.Subset(meal_data, valid_indices)
valid_loader = torch.utils.data.DataLoader(valid_set_balanced,batch_size=batch_size, shuffle=True,num_workers=2)

# train model
input_shape = (train_set_balanced[0][0].shape[0], train_set_balanced[0][0].shape[1])
model_1 = Discriminator( ngpu=1, input_shape =input_shape , out_fea = 1, device=device)
_ = model_1(torch.rand((1, input_shape[0],input_shape[1])))
model_1.to(device)
for p in model_1.parameters():
    p.to(device)
criterion_1 = nn.BCEWithLogitsLoss()
optimizer_1 = optim.Adam(model_1.parameters(),lr=0.01,  weight_decay=0.0)
lrscheduler_1 = optim.lr_scheduler.ReduceLROnPlateau(optimizer_1, mode='min',patience= 2, factor = 0.1,threshold=0.01)
dataloader = (train_loader, valid_loader)
model_1, best_model_1,val_score_1,loss_ls_1, train_acc_ls_1, valid_acc_ls_1 = train_model(model_1,dataloader, optimizer_1, 
                                                                    criterion_1, lrscheduler_1, device= device,
                                                                    n_epochs=30, patience = 5, l1_enabled=True,
                                                                    checkpoint_name ="../models/wenkanw_models/checkpoint_model_resnet.pt" )

Train set size: 2358, with 1179 positive samples and 1179 negative samples
Test set size: 590, with 295 positive samples and 295 negative samples
Training set batch amounts: 24
Test set : 5
Start Training..


/home/wenkanw/.conda/envs/mlenv/lib/python3.8/site-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Epoch: 0,  Epoch_Loss: 1307.0637, Train Acc: 51.2890 %, Train Recall: 0.5991, Validation Acc:  60.6780 %,  Validation Recall: 0.7492  
Checkpoint Saved


Epoch: 1,  Epoch_Loss: 76.4653, Train Acc: 59.1248 %, Train Recall: 0.5882, Validation Acc:  61.0169 %,  Validation Recall: 0.5932  
Checkpoint Saved


Epoch: 2,  Epoch_Loss: 40.0747, Train Acc: 60.7531 %, Train Recall: 0.6180, Validation Acc:  62.5424 %,  Validation Recall: 0.5729  
Checkpoint Saved


Epoch: 3,  Epoch_Loss: 31.1980, Train Acc: 62.9579 %, Train Recall: 0.6472, Validation Acc:  65.4237 %,  Validation Recall: 0.6983  
Checkpoint Saved


Epoch: 4,  Epoch_Loss: 28.4675, Train Acc: 64.2130 %, Train Recall: 0.6988, Validation Acc:  64.7458 %,  Validation Recall: 0.6441  


Epoch: 5,  Epoch_Loss: 29.0773, Train Acc: 64.5183 %, Train Recall: 0.6418, Validation Acc:  64.2373 %,  Validation Recall: 0.6136  


Epoch: 6,  Epoch_Loss: 28.2622, Train Acc: 64.5862 %, Train Recall: 0.6472, Validation Acc:  65.2542 %,  Validation Reca